In [1]:
import optuna
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder
from xgboost import XGBClassifier

In [2]:
# Read dataset
df = pd.read_csv("predictive_maintenance.csv").drop(['Target'], axis=1).dropna().drop_duplicates()

In [3]:
# Apply LabelEncoder to target column
df['Failure Type'] = LabelEncoder().fit_transform(df['Failure Type'])

# Apply OrdinalEncoder to categorical features
categorical_features = df.select_dtypes(include='object').columns
df[categorical_features] = OrdinalEncoder().fit_transform(df[categorical_features])

# Apply MinMaxScaler to continous features
continuous_features = df.select_dtypes(include=['float64', 'int64']).columns
df[continuous_features] = MinMaxScaler().fit_transform(df[continuous_features])

In [4]:
# Remove special characters from column names
df.columns = df.columns.str.replace('[^\w\s]', '', regex=True)

In [5]:
# Split dataset into train and test subsets
X = df.drop(['Failure Type'], axis=1)
y = df['Failure Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Balance dataset
X_train, y_train = SMOTETomek(n_jobs=-1, random_state=42).fit_resample(X_train, y_train)

C:\Users\marce\anaconda3\envs\Cysneiros\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [7]:
# Perform hyperparameter optimization
def objective(trial):
    try:
        eta = trial.suggest_float('eta', 0.01, 0.3, step=0.01)
        max_depth = trial.suggest_int('max_depth', 3, 10)
        subsample = trial.suggest_float('subsample', 0.5, 1.0, step=0.1)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
        min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
        gamma = trial.suggest_float('gamma', 0.0, 1.0, step=0.1)
        reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0, step=0.1)
        reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0, step=0.1)
        clf = XGBClassifier(
            objective='multi:softmax',
            eval_metric='mlogloss',
            eta=eta,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            min_child_weight=min_child_weight,
            gamma=gamma,
            reg_lambda=reg_lambda,
            reg_alpha=reg_alpha,
            n_jobs=-1,
            random_state=42
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        trial.set_user_attr('classification_report', report)
        return accuracy
    except:
        raise optuna.TrialPruned()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1)

[I 2023-06-02 14:08:16,952] A new study created in memory with name: no-name-f566f2a7-ec04-4624-ac0a-b3f08a793466
[I 2023-06-02 14:08:31,348] Trial 12 finished with value: 0.9083333333333333 and parameters: {'eta': 0.16, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 1.0, 'reg_lambda': 0.2, 'reg_alpha': 0.30000000000000004}. Best is trial 12 with value: 0.9083333333333333.
[I 2023-06-02 14:08:31,503] Trial 3 finished with value: 0.8333333333333334 and parameters: {'eta': 0.060000000000000005, 'max_depth': 3, 'subsample': 0.7, 'colsample_bytree': 1.0, 'min_child_weight': 5, 'gamma': 0.8, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.6000000000000001}. Best is trial 12 with value: 0.9083333333333333.
[I 2023-06-02 14:08:31,668] Trial 2 finished with value: 0.749 and parameters: {'eta': 0.01, 'max_depth': 3, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 5, 'gamma': 0.8, 'reg_lambda': 1.0, 'reg_alpha': 0.6000000000000001}. Be

[I 2023-06-02 14:09:14,582] Trial 36 finished with value: 0.9653333333333334 and parameters: {'eta': 0.29000000000000004, 'max_depth': 5, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 3, 'gamma': 0.30000000000000004, 'reg_lambda': 0.5, 'reg_alpha': 0.2}. Best is trial 10 with value: 0.9703333333333334.
[I 2023-06-02 14:09:14,895] Trial 27 finished with value: 0.968 and parameters: {'eta': 0.3, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.5, 'min_child_weight': 2, 'gamma': 0.30000000000000004, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.30000000000000004}. Best is trial 10 with value: 0.9703333333333334.
[I 2023-06-02 14:09:15,051] Trial 26 finished with value: 0.9693333333333334 and parameters: {'eta': 0.3, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.5, 'min_child_weight': 2, 'gamma': 0.30000000000000004, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.30000000000000004}. Best is trial 10 with value: 0.9703333333333334.
[I 2023-06-02 14:0

[I 2023-06-02 14:09:46,427] Trial 47 finished with value: 0.971 and parameters: {'eta': 0.26, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0.30000000000000004, 'reg_lambda': 0.4, 'reg_alpha': 0.1}. Best is trial 34 with value: 0.971.
[I 2023-06-02 14:09:53,007] Trial 55 finished with value: 0.969 and parameters: {'eta': 0.26, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 0.6, 'min_child_weight': 4, 'gamma': 0.4, 'reg_lambda': 0.4, 'reg_alpha': 0.1}. Best is trial 34 with value: 0.971.
[I 2023-06-02 14:09:56,738] Trial 56 finished with value: 0.969 and parameters: {'eta': 0.26, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 0.6, 'min_child_weight': 4, 'gamma': 0.4, 'reg_lambda': 0.4, 'reg_alpha': 0.1}. Best is trial 34 with value: 0.971.
[I 2023-06-02 14:10:05,390] Trial 57 finished with value: 0.969 and parameters: {'eta': 0.26, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 0.6, 'min_child_weight': 4, 'gamma': 0.4, 'r

[I 2023-06-02 14:10:39,747] Trial 78 finished with value: 0.9686666666666667 and parameters: {'eta': 0.23, 'max_depth': 9, 'subsample': 0.8, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.5, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.2}. Best is trial 80 with value: 0.972.
[I 2023-06-02 14:10:40,708] Trial 79 finished with value: 0.9686666666666667 and parameters: {'eta': 0.23, 'max_depth': 9, 'subsample': 0.8, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.5, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.2}. Best is trial 80 with value: 0.972.
[I 2023-06-02 14:10:40,812] Trial 83 finished with value: 0.9683333333333334 and parameters: {'eta': 0.23, 'max_depth': 9, 'subsample': 0.8, 'colsample_bytree': 0.5, 'min_child_weight': 2, 'gamma': 0.2, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.2}. Best is trial 80 with value: 0.972.
[I 2023-06-02 14:10:43,535] Trial 84 finished with value: 0.97 and parameters: {'eta': 0.23, 'max_depth': 9, 'subsample':

[I 2023-06-02 14:11:29,881] Trial 115 finished with value: 0.967 and parameters: {'eta': 0.27, 'max_depth': 10, 'subsample': 0.7, 'colsample_bytree': 0.8, 'min_child_weight': 6, 'gamma': 0.1, 'reg_lambda': 0.0, 'reg_alpha': 0.0}. Best is trial 107 with value: 0.9723333333333334.
[I 2023-06-02 14:11:31,020] Trial 108 finished with value: 0.9713333333333334 and parameters: {'eta': 0.29000000000000004, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.5, 'reg_alpha': 0.0}. Best is trial 107 with value: 0.9723333333333334.
[I 2023-06-02 14:11:31,488] Trial 113 finished with value: 0.9716666666666667 and parameters: {'eta': 0.27, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_weight': 3, 'gamma': 0.1, 'reg_lambda': 0.0, 'reg_alpha': 0.0}. Best is trial 107 with value: 0.9723333333333334.
[I 2023-06-02 14:11:32,449] Trial 114 finished with value: 0.9716666666666667 and parameters: {'eta': 0.27, 'max_depth'

...

[I 2023-06-02 14:33:57,885] Trial 876 finished with value: 0.9726666666666667 and parameters: {'eta': 0.23, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.5, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:34:03,063] Trial 879 finished with value: 0.9756666666666667 and parameters: {'eta': 0.23, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:34:03,347] Trial 878 finished with value: 0.9726666666666667 and parameters: {'eta': 0.23, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:34:03,720] Trial 881 finished with value: 0.971 and parameters

[I 2023-06-02 14:34:41,161] Trial 902 finished with value: 0.9676666666666667 and parameters: {'eta': 0.21000000000000002, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.6000000000000001}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:34:41,495] Trial 904 finished with value: 0.9726666666666667 and parameters: {'eta': 0.21000000000000002, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:34:42,170] Trial 905 finished with value: 0.9733333333333334 and parameters: {'eta': 0.24000000000000002, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:34

[I 2023-06-02 14:35:31,724] Trial 928 finished with value: 0.9726666666666667 and parameters: {'eta': 0.22, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:35:32,792] Trial 929 finished with value: 0.9726666666666667 and parameters: {'eta': 0.22, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:35:34,954] Trial 930 finished with value: 0.9726666666666667 and parameters: {'eta': 0.22, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:35:35,536] Trial 933 finished with value: 0.969 and parameters

[I 2023-06-02 14:36:19,821] Trial 954 finished with value: 0.9723333333333334 and parameters: {'eta': 0.24000000000000002, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.8, 'reg_alpha': 0.2}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:36:20,021] Trial 956 finished with value: 0.9646666666666667 and parameters: {'eta': 0.24000000000000002, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 8, 'gamma': 0.1, 'reg_lambda': 0.8, 'reg_alpha': 0.2}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:36:24,965] Trial 957 finished with value: 0.974 and parameters: {'eta': 0.24000000000000002, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:36:28,287] Trial 958 finished with value: 0.9723333333333334

[I 2023-06-02 14:37:07,080] Trial 981 finished with value: 0.9713333333333334 and parameters: {'eta': 0.23, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 2, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:37:07,238] Trial 980 finished with value: 0.9723333333333334 and parameters: {'eta': 0.22, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 2, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:37:08,666] Trial 982 finished with value: 0.9726666666666667 and parameters: {'eta': 0.22, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 452 with value: 0.9756666666666667.
[I 2023-06-02 14:37:09,527] Trial 984 finished with value: 0.9726666666666667 a

In [8]:
# Print best trial
print(f"\nBest trial: {study.best_trial.params} => {study.best_trial.values[0]:.3f}")
print(f"\n{pd.DataFrame.from_dict(study.best_trial.user_attrs['classification_report']).round(3).T}")


Best trial: {'eta': 0.23, 'max_depth': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1} => 0.976

              precision  recall  f1-score   support
0                 0.846   0.957     0.898    23.000
1                 0.992   0.983     0.988  2903.000
2                 0.773   0.773     0.773    22.000
3                 0.879   0.935     0.906    31.000
4                 0.000   0.000     0.000     7.000
5                 0.114   0.286     0.163    14.000
accuracy          0.976   0.976     0.976     0.976
macro avg         0.601   0.656     0.621  3000.000
weighted avg      0.982   0.976     0.978  3000.000
